<div class="alert alert-info"><center><h1>Биологический нейрон</h1></center></div>

**Нейрон** — электрически возбудимая клетка, которая предназначена для приёма извне, обработки, хранения, передачи и вывода вовне информации с помощью электрических и химических сигналов.  
  
Типичный нейрон состоит из тела клетки, дендритов и одного аксона:

![neuron](images/neuron.png)

Простыми словами можно описать принцип действия нейрона следующим образом:
- через дендриты в нейрон поступают сигналы (раздражители)
- Если комбинация сигналов превышает пороговый уровень - нейрон "выстреливает", т.е. передаёт сигнал дальше через аксон.

Нейроны могут соединяться один с другим, формируя нервные сети.

<div class="alert alert-info"><center><h1>Функции Активации (Activation Functions)</h1></center></div>

Простейшим механизмом активации является **Step activation**, когда перспептрон передаёт на выход значение только в том случае, если сумма взвешенных входящих сигналов больше заданного порога:

![step](images/step_activation.png)

При всей своей простоте, данная функция активации обладает критическим недостатком: она недифференцируемая.  Как результат, она не позволяет осуществлять процесс обучения персептрона.  
  
Для того, чтобы исправить это, было разработано множество других функций активаций, таких как:

![neuron](images/activation_functions.png)

<div class="alert alert-warning"><center><h1>Задание 1 (2 балла)</h1></center></div>

Напишите класс **ActivationFunction** и его подкласс **Sigmoid**, у которого будет функция `forward`, которая:
- будет принимать на вход число и будет сохранять его внутри объекта
- будет возвращать результат в соответствии с фукцией $\sigma(x) = \frac{1}{1 + e^{-x}}$

In [ ]:
import numpy as np


class ActivationFunction:
    ...


class Sigmoid(ActivationFunction):

    def forward(self, value: float) -> float:
        self.value = value
        return 1 / (1 + np.exp(-value))

In [ ]:
sg = Sigmoid()
sg.forward(value = 0)

---

---

<div class="alert alert-info"><center><h1>Персептрон</h1></center></div>

**Персептрон** -  математическая модель биологического нейрона, является базовым элементом нейронных сетей:

![neuron](images/perceptron.png)

**Персептрон** состоит из следующих ключевых элементов:
- `вход` - отвечает за получение входных значений. Является аналогом дендрита биологического нейрона
- `веса` - механизм "важности" входных значений. По аналогии с нейроном - это "толщина" дендрита
- `функция активации` - обрабатывает сумму взвешенных входных сигналов и передаёт результат на выход
- `выход` - отвечает за передачу итогового результата. Аналогичен аксону
  
Практически всегда к входным сигналам также добавляется "bias", который всегда = 1.  
Это позволяет не привязывать выход персептрона к 0 в случае, если все входные сигналы также равны 0 (как в механизме регрессии)

<div class="alert alert-warning"><center><h1>Задание 2 (4 балла)</h1></center></div>

напишите класс **Layer**, у когорого будут следующие входные параметры:
- **n_inputs** - количество входящих значений
- **n_outputs** - количество исходящих значений (в нашем случае = 1)
- **activation** - объект из семейства **ActivationFunction** (в нашем случае - **Sigmoid**)
  
При своём создании объект класса **Layer** должен также создавать атрибут `weights_`, в ктором будут рандомально инициализированны веса для входящих значений, а также для `bias`

Класс **Layer** должен иметь функцию `forward`, принимающую на вход массив *numpy*, и возвращающую результат функции активации (тоже в виде массива).  
Также эта функция должна сохранять полученные на вход значения внутри экземпляра

In [ ]:
import numpy as np


class Layer:
    def __init__(self, n_inputs: int, n_outputs: int = 1, activation = Sigmoid()):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.activation = activation
        self.weights_ = np.random.rand(n_inputs + 1, n_outputs)
        self.inputs_ = None
    
    def forward(self, X: np.ndarray):
        self.inputs_ = np.insert(X, obj = 0, values=1, axis = 0)
        weighted_sum = np.dot(self.inputs_, self.weights_)
        return self.activation.forward(weighted_sum)

In [ ]:
inputs = np.array([0.001, 2, 0.8, 7, 0.6])

In [ ]:
perceptron = Layer(n_inputs = 5, activation=Sigmoid())

perceptron.forward(inputs)

In [ ]:
perceptron.weights_

In [ ]:
perceptron.inputs_

---

<div class="alert alert-warning"><center><h1>Задание 3 (2 балла)</h1></center></div>

напишите класс **LossFunction** и его подкласс **CrossEntropy**, у которого будет функция `loss`, которая будет принимать реальное бинарное значение *y_fact* и вероятность *y_prob* (оба параметра в виде np.array) и будет возвращать результат по формуле:  
  
$$
L = - \sum (y_{fact} * log(y_{prob}) + (1-y_{fact})*log(1-y_{prob}))
$$

In [ ]:
class LossFunction:
    ...

class CrossEntropy (LossFunction):

    def loss(self, y_fact: np.ndarray, y_prob: np.ndarray):
        return -np.sum(y_fact*np.log(y_prob) + (1-y_fact)*np.log(1-y_prob))

In [ ]:
ce = CrossEntropy()

In [ ]:
ce.loss(y_fact = np.array(1), y_prob = np.array(0.363274))

In [ ]:
#проверка
y_fact = np.array([1, 0, 1])  # Реальные значения (бинарные)
y_prob = np.array([0.9, 0.3, 0.4])  # Предсказанные вероятности

In [ ]:
ce.loss(y_fact = y_fact, y_prob = y_prob)

---

<div class="alert alert-info"><center><h1>Обучение. Forward and Backpropagation</h1></center></div>

Процесс обучения персептрона (и в целом нейросети) итеративен и состоит из следующих этапов:
- Сперва персептрон инициализируется с рандомальными весами
- Осуществляется цикл "вперёд":
  - Входные значения перемножаются с соответствующими весами и суммируются
  - Эта сумма подаётся на функцию активации
  - Функция активации возвращает итоговое значение
- Итоговое значение сравнивается с ожидаемым и высчитывается ошибка (Loss)
---
- Осуществляется цикл "назад":
  - при помощи `Chain Rule` рассчитываются частичные производные для всех элементов персептрона
  - исходя из заданного коэффициента обучения (`learning rate`, $\alpha$), веса $w_{i}$ корректируются
- Данный цикл повторяется заданное количество раз или до тех пор, пока итоговая ошибка не опустится ниже заданного порогового значения

![img](images/training.png)

### <center>Chain Rule</center>

Если нам дана функция $y=f(u)$, где $u = g(x)$, то тогда производная этой функции по $x$ будет равно:

$$
\frac{dy}{dx} = \frac{dy}{du}\frac{du}{dx}
$$

Тогда для того, чтобы понять, насколько изменение весов $w$ влияет на изменение $y$ (т.е. производные $\frac{dy}{dw_{i}}$), можно вычислить следующие производные:

![neuron](images/backpropagation.png)

<div class="alert alert-warning"><center><h1>Задание 4 (8 баллов)</h1></center></div>

Модифицируйте классы **Layer**, **LossFuncton** и **ActivationFunction** таким образом, чтобы можно было рассчитать их частичные производные, и добавьте функцию `back`, позволяющую осуществить backpropagation.

<div class="alert alert-danger"><center><h3>Это задание очень сложное, и даже частичное его выполнение будет учитываться</h3></center></div>

In [2]:
import numpy as np

In [3]:
class ActivationFunction():
    ...

    
class Sigmoid(ActivationFunction):
    
    def __init__(self):
        self.value = None
        
    def forward(self, value):
        self.value = value
        return 1 / (1 + np.exp(-value))
    
    def gradient(self, value):
        sigmoid = self.forward(value)
        return sigmoid * (1 - sigmoid) 

In [4]:
class LossFunction():
    ...
    
    
class CrossEntropy(LossFunction):

    def loss(self, y_fact: np.ndarray, y_prob: np.ndarray):
        y_prob = np.clip(y_prob, 1e-15, 1 - 1e-15)
        loss = -np.sum(y_fact * np.log(y_prob) + (1 - y_fact) * np.log(1 - y_prob))
        return loss
    
    def gradient(self, y_fact: np.ndarray, y_prob: np.ndarray):
        y_prob = np.clip(y_prob, 1e-15, 1 - 1e-15)
        return -(y_fact/ y_prob) + (1 - y_fact) / (1 - y_prob)

In [5]:
rng = np.random.default_rng(seed = 42)

class Layer():
    def __init__(self, n_inputs: int, n_outputs: int = 1, activation = Sigmoid()):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.activation = activation
        self.weights_ = rng.random(size = (n_inputs + 1, n_outputs))
        self.inputs_ = None
        self.outputs_ = None
        self.weighted_sum_ = None #для проверки 
        self.distributed_gradient_ = None # для проверки
        
    def forward(self, X: np.ndarray) -> np.ndarray:
        self.inputs_ = np.insert(X, obj = 0, values = 1, axis = 1) # добавляет 1 к входным значениям (для bias) 
        self.weighted_sum_ = np.dot(self.inputs_, self.weights_)
        self.outputs_ = activation.forward(self.weighted_sum_)
        return self.outputs_
    
    def backward(self, loss_func_gradient: float, learning_rate: float = 0.2):
        # расчитываем производную Activation Function (производная loss_func расчитана внешне = loss_func_gradient)
        gradient = activation.gradient(self.weighted_sum_) * loss_func_gradient
        # распределяем градиент
        self.distributed_gradient_ = np.dot(gradient, self.inputs_) / self.inputs_.shape[0]
        # обновляем веса 
        self.weights_ -= self.distributed_gradient_.T * learning_rate
        return gradient

In [16]:
activation = Sigmoid()
loss_function = CrossEntropy()
layer = Layer(n_inputs=8, n_outputs=1, activation=activation)

In [17]:
X = np.array([[2, 1.1, 7, 0.8, 0.34, 0.6, 5, 1.72]])
y_true = np.array([[0]])

In [18]:
layer.weights_

array([[0.22723872],
       [0.55458479],
       [0.06381726],
       [0.82763117],
       [0.6316644 ],
       [0.75808774],
       [0.35452597],
       [0.97069802],
       [0.89312112]])

In [19]:
loses = []
for epoch in range(50): 
    y_prob = layer.forward(X)
    loss = loss_function.loss(y_fact = y_true, y_prob = y_prob)
    
    loses.append(loss)
    print(loss)
    
    gradient = loss_function.gradient(y_fact = y_true, y_prob= y_prob)
    update = layer.backward(gradient)

14.56548133654328
0.09634034989477644
0.021275844889491642
0.014967507995754276
0.011671011691657882
0.009607551311503088
0.00818321850736742
0.007136550455546433
0.006332915009660183
0.0056954313288833875
0.005176815040234264
0.004746304875185894
0.004382986098957274
0.004072122446547191
0.00380302045107704
0.0035677256371603972
0.0033601944997189233
0.003175750754049025
0.0030107178534770408
0.002862164355264543
0.0027277235729997705
0.0026054633430051335
0.0024937903399746314
0.0023913786777026275
0.002297115880280648
0.002210061475255103
0.0021294148905796977
0.002054490299709665
0.001984696718144806
0.0019195221130308
0.0018585206108195965
0.0018013021192562761
0.001747523847397658
0.0016968833299844163
0.0016491126532553052
0.0016039736471412144
0.0015612538599800935
0.0015207631708632418
0.0014823309246318251
0.0014458034976740312
0.0014110422206903275
0.0013779215987273197
0.0013463277799381321
0.0013161572333901265
0.0012873156033221382
0.0012597167129421772
0.0012332816954506

In [10]:
layer.weights_

array([[ 0.51905498],
       [-0.0709237 ],
       [ 0.57820674],
       [-1.08693947],
       [-0.10974351],
       [ 0.88895599]])

----

### Пошагово

In [ ]:
l1 = layer.forward(X)
l1

In [ ]:
layer.weights_

In [ ]:
lf_loss = loss_function.loss(y_fact = y_true, y_prob=l1)
lf_loss

In [ ]:
lf_gradient = loss_function.gradient(y_fact = y_true, y_prob=l1)
lf_gradient

In [ ]:
l1_back = layer.backward(loss_func_gradient=lf_gradient)
l1_back

In [ ]:
layer.weights_

In [ ]:
l1_2 = layer.forward(X)
l1_2

In [ ]:
lf_loss_2 = loss_function.loss(y_fact = y_true, y_prob=l1_2)
lf_loss_2

In [ ]:
lf_gradient_2 = loss_function.gradient(y_fact = y_true, y_prob=l1_2)
lf_gradient_2

In [ ]:
l1_back_2 = layer.backward(loss_func_gradient=lf_gradient_2)
l1_back_2

In [ ]:
layer.weights_

In [ ]:
loses = []

---

# <center>Удачи!</center>